In [179]:
import os
import pandas as pd
import numpy as np
import urllib
import plotly.graph_objects as go
import calendar

In [180]:
path = r'C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\TempComparison\resample'
os.chdir(path)
csv = path + r'\csv.csv'
f = open(csv,'w+')

In [181]:
urllib.request.urlretrieve(
    'http://coagmet.colostate.edu/cgi-bin/web_services.pl?type=daily&sids=bnv01' + 
    '&sdate=2010-10-12&edate=2018-12-31&elems=tmax,mxtm,tmin,mntm',filename=csv)
df = pd.read_csv(csv)
df = df.reset_index()

In [182]:
df.columns = ['station','date','tmax','mxtm','tmin','mntm']

df['tmax'] = (32 + (df.tmax * (9/5)))
df['tmin'] = (32 + (df.tmin * (9/5)))

In [183]:
df['tmax_time'] = (df.date + ' ' + df.mxtm)
df['tmin_time'] = (df.date + ' ' + df.mntm)

In [184]:
df['tmax_time'] = pd.to_datetime(df.tmax_time)
df['tmin_time'] = pd.to_datetime(df.tmin_time)

In [185]:
df['date'] = df.tmax_time.dt.date
df['date'] = pd.to_datetime(df.date)

In [186]:
tmin = ['station','tmin','tmin_time','date']
tmin_df = df[tmin]

tmax = ['station','tmax','tmax_time','date']
tmax_df = df[tmax]

In [187]:
tmax_resample = tmax_df.resample(rule='24H',label='right',on='tmax_time',base=17).max()
tmin_resample = tmin_df.resample(rule='24H',label='right',on='tmin_time',base=17).min()

In [188]:
stuff = ['tmin_time']
tmin_resample = tmin_resample.drop(stuff,axis=1)
tmin_resample = tmin_resample.reset_index()
tmin_resample = tmin_resample.drop(stuff,axis=1)

In [189]:
stuff = ['tmax_time']
tmax_resample = tmax_resample.drop(stuff,axis=1)
tmax_resample = tmax_resample.reset_index()
tmax_resample = tmax_resample.drop(stuff,axis=1)

In [190]:
tmin_resample.dtypes

station            object
tmin              float64
date       datetime64[ns]
dtype: object

In [191]:
urllib.request.urlretrieve(
    'http://data.rcc-acis.org/StnData?sid=051071&sdate=2010-10-12&edate=2018-12-31' + 
    '&elems=mint&output=csv',filename=csv)
coop_mint_df = pd.read_csv(csv)
coop_mint_df = coop_mint_df.reset_index()
coop_mint_df.columns = ['date','mint']
coop_mint_df = coop_mint_df.replace('M',np.nan,regex=True)

In [192]:
urllib.request.urlretrieve(
    'http://data.rcc-acis.org/StnData?sid=051071&sdate=2010-10-12&edate=2018-12-31' + 
    '&elems=maxt&output=csv',filename=csv)
coop_maxt_df = pd.read_csv(csv)
coop_maxt_df = coop_maxt_df.reset_index()
coop_maxt_df.columns = ['date','maxt']
coop_maxt_df = coop_maxt_df.replace('M',np.nan,regex=True)

In [193]:
tmin_comp_df = tmin_resample.merge(coop_mint_df,left_index=True,right_index=True)
tmax_comp_df = tmax_resample.merge(coop_maxt_df,left_index=True,right_index=True)

In [194]:
tmax_comp_df['maxt'] = tmax_comp_df.maxt.astype(float)
tmin_comp_df['mint'] = tmin_comp_df.mint.astype(float)

In [195]:
tmax_comp_df['diff'] = (tmax_comp_df.tmax - tmax_comp_df.maxt)
tmin_comp_df['diff'] = (tmin_comp_df.tmin - tmin_comp_df.mint)

In [196]:
stuff = ['date_y']
tmax_comp_df = tmax_comp_df.drop(stuff,axis=1)
tmin_comp_df = tmin_comp_df.drop(stuff,axis=1)

In [197]:
tmin_comp_df.columns = ['station','tmin','date','mint','diff']
tmax_comp_df.columns = ['station','tmax','date','maxt','diff']

In [198]:
tmax_comp_df = tmax_comp_df.dropna()
tmin_comp_df = tmin_comp_df.dropna()
tmin_comp_df['month'] = tmin_comp_df.date.dt.month
tmax_comp_df['month'] = tmax_comp_df.date.dt.month
tmin_comp_df['month'] = tmin_comp_df.month.astype(int)
tmax_comp_df['month'] = tmax_comp_df.month.astype(int)

In [199]:
tmin_comp_df.to_csv(path + r'\min\bnv01_coop_tmincomp.csv')
tmax_comp_df.to_csv(path + r'\max\bnv01_coop_tmaxcomp.csv')

In [22]:
tmin_comp_df['month'] = tmin_comp_df['month'].apply(lambda x: calendar.month_abbr[x])
tmax_comp_df['month'] = tmax_comp_df['month'].apply(lambda x: calendar.month_abbr[x])

In [24]:
tmax_comp_df

,station,tmax,date,maxt,diff,month
0,rfd01,85.3520,1992-06-05,84.0,1.3520,Jun
1,rfd01,76.2800,1992-06-06,77.0,-0.7200,Jun
3,rfd01,84.9560,1992-06-08,83.0,1.9560,Jun
5,rfd01,80.6360,1992-06-10,80.0,0.6360,Jun
6,rfd01,77.9180,1992-06-11,79.0,-1.0820,Jun
7,rfd01,80.7440,1992-06-12,81.0,-0.2560,Jun
9,rfd01,88.6820,1992-06-14,88.0,0.6820,Jun
10,rfd01,87.2780,1992-06-15,89.0,-1.7220,Jun
11,rfd01,87.3140,1992-06-16,90.0,-2.6860,Jun
12,rfd01,84.3980,1992-06-17,86.0,-1.6020,Jun


In [18]:
tmax_fig = go.Figure(data=[go.Bar()],
                    layout=go.Layout(title=go.layout.Title(text='Average Monthly Max Temp Comparison'),
                                    xaxis=go.layout.XAxis(title=go.layout.xaxis.Title(text='Month')),
                                    yaxis=go.layout.YAxis(title=go.layout.yaxis.Title(text='Temperature (°F)'))))
tmax_fig.add_trace(
    go.Bar(x=monthly.month,y=monthly.tmax,text=monthly.tmax_diff,name='RFD01'))

tmax_fig.add_trace(
    go.Bar(x=monthly.month,y=monthly.maxt,text=monthly.tmax_diff,name='Rocky Ford 2 ESE'))

tmax_fig.show()

AttributeError: 'DataFrame' object has no attribute 'tmax_diff'

In [19]:
df

,station,date,tmax,mxtm,tmin,mntm,tmax_time,tmin_time,month
0,rfd01,1992-06-04,76.2620,18:41:00,52.8980,01:24:00,1992-06-04 18:41:00,1992-06-04 01:24:00,6.0
1,rfd01,1992-06-05,85.3520,14:24:00,53.8880,05:40:00,1992-06-05 14:24:00,1992-06-05 05:40:00,6.0
2,rfd01,1992-06-06,76.2800,15:27:00,47.0840,05:34:00,1992-06-06 15:27:00,1992-06-06 05:34:00,6.0
3,rfd01,1992-06-07,84.9560,17:21:00,52.7900,06:22:00,1992-06-07 17:21:00,1992-06-07 06:22:00,6.0
4,rfd01,1992-06-08,76.4240,15:03:00,54.2300,02:57:00,1992-06-08 15:03:00,1992-06-08 02:57:00,6.0
5,rfd01,1992-06-09,68.5220,17:36:00,51.4040,00:00:00,1992-06-09 17:36:00,1992-06-09 00:00:00,6.0
6,rfd01,1992-06-10,80.6360,16:46:00,46.2200,05:21:00,1992-06-10 16:46:00,1992-06-10 05:21:00,6.0
7,rfd01,1992-06-11,77.9180,15:00:00,53.0420,18:18:00,1992-06-11 15:00:00,1992-06-11 18:18:00,6.0
8,rfd01,1992-06-12,80.7440,16:53:00,51.4220,05:43:00,1992-06-12 16:53:00,1992-06-12 05:43:00,6.0
9,rfd01,1992-06-13,88.6820,17:02:00,56.9480,04:11:00,1992-06-13 17:02:00,1992-06-13 04:11:00,6.0
